This notebook implements the assembly of the libre-solar OSH project:

Currently includes only the frame of the enclosure


# Initialisation

In [ ]:
# The following should reload if_lib if we change it on disk
%load_ext autoreload
%aimport if_lib, if_utils, if_dpp, if_graphics
%autoreload 1
import os
import json
import random

from if_utils import get_filename, show_data, save_traces

from if_lib import generate_random_challenge, read_HMAC, read_keypair, get_id_person, get_location_id, \
get_unit_id, get_resource_spec_id, get_resource, get_process, create_event, make_transfer, reduce_resource, set_user_location

from if_groups import get_processgrp, insert_procingrp

from if_dpp import trace_query, check_traces, er_before, get_dpp

from if_prop import get_proposal, get_intent, get_proposedIntent, show_proposal, get_satisfaction, check_proposals

from if_graphics import vis_dpp, make_sankey, consol_trace


In [ ]:
USE_CASE = 'libre-solar'

# What endpoint are we talking to?
# debug
# ENDPOINT = 'http://65.109.11.42:10000/api'
ENDPOINT = 'https://zenflows.interfacer-staging.dyne.im/api'
# staging
# ENDPOINT = 'http://65.109.11.42:8000/api'
# ENDPOINT = 'https://zenflows-staging.interfacer.dyne.org/api'
# testing
# ENDPOINT = 'http://65.109.11.42:9000/api'
# ENDPOINT = 'https://zenflows-test.interfacer.dyne.org/api'

USERS = ['one', 'two', 'Michel', 'Adam']
# USERS = ['one', 'two']

In [ ]:
# Calculate names of settings files
USERS_FILE = get_filename('cred_users.json', ENDPOINT, USE_CASE)
LOCS_FILE = get_filename('loc_users.json', ENDPOINT, USE_CASE)
UNITS_FILE = get_filename('units_data.json', ENDPOINT, USE_CASE)
SPECS_FILE = get_filename('res_spec_data.json', ENDPOINT, USE_CASE)

In [ ]:
# Read or define user data that is going to be used in the GraphQL calls


# create data structure to hold processes
process_data = {}

# create data structure to hold process groups
processgrp_data = {}

# create data structure to hold proposals
proposal_data = {}

# create data structure to hold intents
intent_data = {}

# create data structure to hold proposed intents
prop_int_data = {}

# create data structure to hold satisfactions
satisfaction_data = {}

# create data structures to hold resources and events (possibly to compare results from track and trace)
res_data = {}
event_seq = []


if os.path.isfile(USERS_FILE):
    with open(USERS_FILE,'r') as f:
        users_data = json.loads(f.read())
    print("Credentials file available for users")
else:
    users_data = {}
    users_data['one'] = {
      "userChallenges": {
        "whereParentsMet": "London",
        "nameFirstPet": "Fuffy",
        "nameFirstTeacher": "Jim",
        "whereHomeTown": "Paris",
        "nameMotherMaid": "Wright"
      },
      "name": "User One",
      "username": "user1_username",
      "email": "user1@example.org",
      "note": "me.user1.org"
    }
    users_data['two'] = {
        "userChallenges": {
            "whereParentsMet":"Amsterdam",
            "nameFirstPet":"Toby",
            "nameFirstTeacher":"Juliet",
            "whereHomeTown":"Rome",
            "nameMotherMaid":"Banks"
        },
        "name": "User Two",
        "username": "user2",
        "email": "user2@example.org",
        "note" : "me.user2.org"
    }
    users_data['Michel'] = {
      "userChallenges": {
        "whereParentsMet": "London",
        "nameFirstPet": "Fuffy",
        "nameFirstTeacher": "Jim",
        "whereHomeTown": "Paris",
        "nameMotherMaid": "Shortshovel"
      },
      "name": "Michel Langhammer",
      "username": "michel",
      "email": "michel@fabcity.hamburg",
      "note": "michel@fabcity.hamburg - test account"
    }
    users_data['Adam'] = {
        "userChallenges": {
            "whereParentsMet":"Amsterdam",
            "nameFirstPet":"Toby",
            "nameFirstTeacher":"Juliet",
            "whereHomeTown":"Rome",
            "nameMotherMaid":"Maiden"
        },
        "name": "Adam Burns",
        "username": "adam",
        "email": "adamb@networkcommons.org",
        "note" : "adamb@networkcommons.org - test account"
    }

    with open(USERS_FILE,'w') as f:
        json.dump(users_data, f)


if os.path.isfile(LOCS_FILE):
    with open(LOCS_FILE,'r') as f:
        locs_data = json.loads(f.read())
    print("Location file available")
else:
    locs_data = {}
    locs_data['one'] = {
        "name": "FabCity Hamburg",
        "lat" : 53.54651094994547,
        "long": 9.996357255816038,
        "addr": "Zippelhaus 5A, 20457 Hamburg, Germany",
        "note": "location.user1.org"
    }
    locs_data['two'] = {
        "name": "Fab City Hamburg",
        "lat" : 53.54651094994547,
        "long" : 9.996357255816038,
        "addr" : "Zippelhaus 5A, 20457 Hamburg, Germany",
        "note": "location.user2.org"
    }
    locs_data['Michel'] = {
        "name": "Fab City Hamburg",
        "lat" : 53.54651094994547,
        "long" : 9.996357255816038,
        "addr" : "Zippelhaus 5A, 20457 Hamburg, Germany",
        "note": "location.user2.org"
    }
    locs_data['Adam'] = {
        "name": "Fab City Hamburg",
        "lat" : 53.54651094994547,
        "long" : 9.996357255816038,
        "addr" : "Zippelhaus 5A, 20457 Hamburg, Germany",
        "note": "location.user2.org"
    }

    with open(LOCS_FILE,'w') as f:
        json.dump(locs_data, f)


if os.path.isfile(UNITS_FILE):
    with open(UNITS_FILE,'r') as f:
        units_data = json.loads(f.read())
    print(f"Unit file available")
else:
    units_data = {}
#     with open(file,'w') as f:
#         json.dump(units_data, f)



if os.path.isfile(SPECS_FILE):
    with open(SPECS_FILE,'r') as f:
        res_spec_data = json.loads(f.read())
    print(f"Resource Spec file available")
else:
    res_spec_data = {}
  

In [ ]:
# Read HMAC or get it from the server
for user in USERS:
    read_HMAC(USERS_FILE, users_data, user, endpoint=ENDPOINT)

In [ ]:
# Read the keypair
for user in USERS:
    read_keypair(USERS_FILE, users_data, user)

In [ ]:
# Read or get id of the person
for user in USERS:
    get_id_person(USERS_FILE, users_data, user, endpoint=ENDPOINT)

In [ ]:
# Read or get the location id
for user in USERS:
    get_location_id(LOCS_FILE, users_data[user], locs_data, user, endpoint=ENDPOINT)
    set_user_location(USERS_FILE, users_data, locs_data, user, endpoint=ENDPOINT)

In [ ]:
# Get the ids of all units
get_unit_id(UNITS_FILE, users_data['two'], units_data, 'piece', 'u_piece', 'om2:one', endpoint=ENDPOINT)
get_unit_id(UNITS_FILE, users_data['two'], units_data, 'mass', 'kg', 'om2:kilogram', endpoint=ENDPOINT)
get_unit_id(UNITS_FILE, users_data['two'], units_data, 'volume', 'lt', 'om2:litre', endpoint=ENDPOINT)
get_unit_id(UNITS_FILE, users_data['one'], units_data, 'time', 'h', 'om2:hour', endpoint=ENDPOINT)


# Resource Specifications

In [ ]:
# RESOURCE SPECIFICATIONS
# Read all the resource specifications
name = 'Allen_Key'
note = 'Specification for DIN911 4mm hex tool (aka Allen Key) for assembly'
classification = 'https://www.fasteners.eu/standards/DIN/911/'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'M6nutplate'
note = 'Specification for M6 nut for Right-Angle-Connector '
classification = 'https://www.fasteners.eu/standards/DIN/562/'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'M6x12'
note = 'Specification for screw for right angled connector'
classification = 'https://www.fasteners.eu/standards/ISO/7380/'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'M6x25cyl'
note = 'Specification for screw for direct corner connection'
classification = 'https://www.fasteners.eu/standards/DIN/7984/'
default_unit_id = units_data['mass']['id']
get_resource_spec_id(SPECS_FILE, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'RAConnector'
note = 'Specification for connector'
classification = 'A resource specification URI'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'Tslot16'
note = 'Specification for a frame slot'
classification = 'https://www.motedis.com/en/Profile-20x20-B-type-slot-6'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'Tslot32'
note = 'Specification for a frame slot'
classification = 'https://www.motedis.com/en/Profile-20x20-B-type-slot-6'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'Tslot32con'
note = 'Specification for a frame slot modifed by designer'
classification = 'https://www.motedis.com/en/Profile-20x20-B-type-slot-6'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'assembly_construction'
note = 'Specification for Frame assembly work'
classification = 'A resource specification URI'
default_unit_id = units_data['time']['id']
get_resource_spec_id(SPECS_FILE, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'upper_frame'
note = 'Specification for upperframe'
classification = 'A resource specification URI'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'lower_frame'
note = 'Specification for lowerframe'
classification = 'A resource specification URI'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'pillar'
note = 'Specification for pillar'
classification = 'A resource specification URI'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'box_frame'
note = 'Specification for box_frame'
classification = 'A resource specification URI'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)



# Resource Creation & Assignment

In [ ]:
# RESOURCES
# We create the resources that will not be saved to file as it is assumed they are recreated at each run

res_name = 'Allen_Key'
amount = 1

get_resource(res_data, res_spec_data, res_name, users_data['two'], event_seq, amount, endpoint=ENDPOINT)

In [ ]:
res_name = 'M6nutplate'
amount = 50

get_resource(res_data, res_spec_data, res_name, users_data['two'], event_seq, amount, endpoint=ENDPOINT)

In [ ]:
res_name = 'M6x12'
amount = 50

get_resource(res_data, res_spec_data, res_name, users_data['two'], event_seq, amount, endpoint=ENDPOINT)

In [ ]:
res_name = 'M6x25cyl'
amount = 50

get_resource(res_data, res_spec_data, res_name, users_data['two'], event_seq, amount, endpoint=ENDPOINT)

In [ ]:
res_name = 'RAConnector'
amount = 50

get_resource(res_data, res_spec_data, res_name, users_data['two'], event_seq, amount, endpoint=ENDPOINT)

In [ ]:
res_name = 'Tslot16'
amount = 4

get_resource(res_data, res_spec_data, res_name, users_data['two'], event_seq, amount, endpoint=ENDPOINT)

In [ ]:
res_name = 'Tslot32'
amount = 50

get_resource(res_data, res_spec_data, res_name, users_data['two'], event_seq, amount, endpoint=ENDPOINT)

In [ ]:
res_name = 'Tslot32con'
amount = 50

get_resource(res_data, res_spec_data, res_name, users_data['two'], event_seq, amount, endpoint=ENDPOINT)

# Process Object Definitions

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the upper frame
# print(process_data)
process_name = 'UpperFrame'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the upper frame
# print(process_data)
process_name = 'UpperFrame_1'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the upper frame
# print(process_data)
process_name = 'UpperFrame_2'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the upper frame
# print(process_data)
process_name = 'UpperFrame_3'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the upper frame
# print(process_data)
process_name = 'UpperFrame_4'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the upper frame
# print(process_data)
process_name = 'LowerFrame'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the upper frame
# print(process_data)
process_name = 'LowerFrame_1'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the upper frame
# print(process_data)
process_name = 'LowerFrame_2'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the upper frame
# print(process_data)
process_name = 'LowerFrame_3'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the upper frame
# print(process_data)
process_name = 'LowerFrame_4'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the Pillar
# print(process_data)
process_name = 'Pillar'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the Pillar
# print(process_data)
process_name = 'Pillar_1'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the Pillar
# print(process_data)
process_name = 'Pillar_2'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the Pillar
# print(process_data)
process_name = 'First_Pillar_1'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the Pillar
# print(process_data)
process_name = 'First_Pillar_2'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the Pillar
# print(process_data)
process_name = 'Second_Pillar_1'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the Pillar
# print(process_data)
process_name = 'Second_Pillar_2'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the Pillar
# print(process_data)
process_name = 'Third_Pillar_1'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the Pillar
# print(process_data)
process_name = 'Third_Pillar_2'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the Pillar
# print(process_data)
process_name = 'Forth_Pillar_1'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the Pillar
# print(process_data)
process_name = 'Forth_Pillar_2'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the Lower Frame & a Pillar
# print(process_data)
process_name = 'Pillar_Lower_Frame_Connection'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the Box Frame & First Pillar
# print(process_data)
process_name = 'First_Pillar_Box_Frame_Connect'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the Box Frame & Second Pillar
# print(process_data)
process_name = 'Second_Pillar_Box_Frame_Connect'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the  Lower Frame & Third Pillar
# print(process_data)
process_name = 'Third_Pillar_Box_Frame_Connect'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the  Lower Frame & Forth Pillar
# print(process_data)
process_name = 'Forth_Pillar_Box_Frame_Connect'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the Side Cover Nutplates Insertion
# print(process_data)
process_name = 'Box_Frame_Side_Cover_Nutplates_Insertion'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the Side Cover Nutplates Insertion
# print(process_data)
process_name = 'Box_Frame_Front_Back_Cover_Nutplates_Insertion'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the Side Cover Nutplates Insertion
# print(process_data)
process_name = 'Box_Frame_Mountplate_Nutplates_Insertion'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the Side Cover Nutplates Insertion
# print(process_data)
process_name = 'Box_Upper_Frame_Connection'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

# Process Input & Output Event Definitions

## UpperFrame

In [ ]:
# UpperFrame 1
# Input event 1
# Define input consume Tslot32con event for UpperFrame Assembly: 
process_name = 'UpperFrame_1'
event_note='Input consume Tslot32con event for Upper Frame Assembly'

action = 'consume'
amount = 1
resource = 'Tslot32con'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# UpperFrame 1
# Input event 2
# Define input consume Tslot16 event for UpperFrame Assembly: 
process_name = 'UpperFrame_1'
event_note='Input consume Tslot16 event for Upper Frame Assembly'

action = 'consume'
amount = 1
resource = 'Tslot16'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# UpperFrame 1
# Input event 3
# Define input consume M6x25cyl event for UpperFrame Assembly: 
process_name = 'UpperFrame_1'
event_note='Input consume M6x25cyl event for Upper Frame Assembly'

action = 'consume'
amount = 1
resource = 'M6x25cyl'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# UpperFrame 1
# Input event 4
# Define input use Allen Key for UpperFrame Assembly: 
process_name = 'UpperFrame_1'
event_note='Input use Allen Key event for Upper Frame Assembly'

action = 'use'
amount = 1
resource = 'Allen_Key'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

#####
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
# effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.03


event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# UpperFrame 1
# Input event 5
# Define input event work with using Allen Key to assemble UpperFrame_1
process_name = 'UpperFrame_1'
event_note='Make a corner junction between a Tslot32con and Tslot16 by a M6x25cyl screw.'

action = 'work'
amount = 0.05
agent = 'one'

cur_pros = process_data[f'{process_name}']

effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = amount


event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# PROCESS wrap
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# UpperFrame 1
# Output event 1
# Define output event produce for UpperFrame
process_name = 'UpperFrame_1'
event_note='produce UpperFrame'

action = 'produce'
amount = 1
resource = 'upper_frame'
agent = 'two'

cur_pros = process_data[f'{process_name}']

res_data[f'{resource}_res'] = {
    "res_ref_id": f'upper_frame-{random.randint(0, 10000)}',
    "name": 'upper_frame',
    "spec_id": res_spec_data['upper_frame']['id']
}
cur_res = res_data[f'{resource}_res']

# for produce events, note new_res=cur_res parameter is required
event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, new_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# UpperFrame 2
# Input event 1
# Define input event accept for UpperFrame 1
process_name = 'UpperFrame_2'
event_note='accept UpperFrame to be modified'

action = 'accept'
amount = 1
resource = 'upper_frame'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# UpperFrame 2
# Input event 2
# Define input consume 2 x M6nutplates event for UpperFrame Assembly: 
process_name = 'UpperFrame_2'
event_note='Input consume 2 x M6nutplate event for Upper Frame Assembly'

action = 'consume'
amount = 2
resource = 'M6nutplate'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# UpperFrame 2
# Input event 3
# Define input event work inserting 2 x M6nutplates into the upper and the outer slot of the Tslot16 beam of the Upper Frame
process_name = 'UpperFrame_2'
event_note='Insert two M6nutplates into the upper and the outer slot of the Tslot16 beam of the Upper Frame.'

action = 'work'
# amount = 0.05
agent = 'one'

cur_pros = process_data[f'{process_name}']
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.05

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# PROCESS wrap
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# UpperFrame 2
# Output event 1
# Define output modify Upper Frame
process_name = 'UpperFrame_2'
event_note='modify UpperFrame_2'

action = 'modify'
amount = 1
resource = 'upper_frame'
agent = 'two'
cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']


event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# UpperFrame 3
# Input event 1
# Define input event accept for UpperFrame 2
process_name = 'UpperFrame_3'
event_note='accept UpperFrame to be modified'

action = 'accept'
amount = 1
resource = 'upper_frame'
agent = 'two'


cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# UpperFrame 3
# Input event 2
# Define input consume 1 x M6nutplate event for UpperFrame Assembly
process_name = 'UpperFrame_3'
event_note='Input consume 1 x M6nutplate event for Upper Frame Assembly'

action = 'consume'
amount = 1
resource = 'M6nutplate'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# UpperFrame 3
# Input event 3
# Define input consume 1 x Tslot16 event for UpperFrame Assembly
process_name = 'UpperFrame_3'
action = 'consume'
event_note='Input consume 1 x Tslot16 event for Upper Frame Assembly'
amount = 1
resource = 'Tslot16'
agent = 'two'
cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})
# event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# UpperFrame 3
# Input event 4
# Define input consume 1 x M6x25cyl event for UpperFrame Assembly
process_name = 'UpperFrame_3'
action = 'consume'
event_note='Input consume 1 x M6x25cyl event for Upper Frame Assembly'
amount = 1
resource = 'M6x25cyl'
agent = 'two'
cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# UpperFrame 3
# Input event 5
# Define input use Allen Key for UpperFrame Assembly: 
process_name = 'UpperFrame_3'
event_note='Input use Allen Key event for Upper Frame Assembly'

action = 'use'
amount = 1
resource = 'Allen_Key'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

#####
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
# effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.03


event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# UpperFrame 3
# Input event 6
# Define input event work of the Upper Frame
process_name = 'UpperFrame_3'
event_note='Before connecting the second Tslot16 beam you must insert a nutplate into the outer slot of the long Tslot32con, which will be enclosed by the M6x25cyl during the corner connection.'

action = 'work'
# amount = 0.05
agent = 'one'

cur_pros = process_data[f'{process_name}']
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.05

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# PROCESS wrap
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# UpperFrame 3
# Output event 1
# Define output modify Upper Frame
process_name = 'UpperFrame_3'
event_note='modify UpperFrame_3'

action = 'modify'
amount = 1
resource = 'upper_frame'
agent = 'two'


cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# UpperFrame 4
# Input event 1
# Define input event accept for UpperFrame 2
process_name = 'UpperFrame_4'
event_note='accept UpperFrame to be modified'

action = 'accept'
amount = 1
agent = 'two'

cur_pros = process_data[f'{process_name}']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# UpperFrame 4
# Input event 2
# Define input consume 3 x M6nutplate event for UpperFrame Assembly
process_name = 'UpperFrame_4'
event_note='Input consume 3 x M6nutplate event for Upper Frame Assembly'

action = 'consume'
amount = 3
resource = 'M6nutplate'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# EVENTS
# UpperFrame 4
# Input event 3
# Define input consume Tslot32con event for UpperFrame Assembly: 
process_name = 'UpperFrame_4'
event_note='Input consume Tslot32con event for Upper Frame Assembly'

action = 'consume'
amount = 2
resource = 'Tslot32con'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# UpperFrame 4
# Input event 4
# Define input consume M6x25cyl event for UpperFrame Assembly: 
process_name = 'UpperFrame_4'
event_note='Input consume M6x25cyl event for Upper Frame Assembly'

action = 'consume'
amount = 2
resource = 'M6x25cyl'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# UpperFrame 4
# Input event 5
# Define input use Allen Key for UpperFrame Assembly: 
process_name = 'UpperFrame_4'
event_note='Input use Allen Key event for Upper Frame Assembly'

action = 'use'
amount = 1
resource = 'Allen_Key'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

#####
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
# effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.03


event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# UpperFrame 4
# Input event 6
# Define input use Allen Key for UpperFrame Assembly: 
process_name = 'UpperFrame_4'
event_note='Input use Allen Key event for Upper Frame Assembly'

action = 'use'
amount = 1
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

#####
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
# effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.03


event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# PROCESS wrap
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# UpperFrame 4
# Output event 1
# Define output modify Upper Frame
process_name = 'UpperFrame_4'
event_note='modify UpperFrame_4'

action = 'modify'
amount = 1
resource = 'upper_frame'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']


event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


## LowerFrame

In [ ]:
# EVENT
# LowerFrame 1
# Input event 1
# Define input consume Tslot32con event for LowerFrame Assembly: 
process_name = 'LowerFrame_1'
event_note='Input consume Tslot32con event for Lower Frame Assembly'

action = 'consume'
amount = 1
resource = 'Tslot32con'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# LowerFrame 1
# Input event 2
# Define input consume Tslot16 event for LowerFrame Assembly: 
process_name = 'LowerFrame_1'
event_note='Input consume Tslot16 event for Lower Frame Assembly'

action = 'consume'
amount = 1
resource = 'Tslot16'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# LowerFrame 1
# Input event 3
# Define input consume M6x25cyl event for LowerFrame Assembly: 
process_name = 'LowerFrame_1'
event_note='Input consume M6x25cyl event for Lower Frame Assembly'

action = 'consume'
amount = 1
resource = 'M6x25cyl'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# LowerFrame 1
# Input event 4
# Define input use Allen Key for LowerFrame Assembly: 
process_name = 'LowerFrame_1'
event_note='Input use Allen Key event for Lower Frame Assembly'

action = 'use'
amount = 1
resource = 'Allen_Key'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

#####
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
# effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.03


event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# LowerFrame 1
# Input event 5
# Define input event work of the Lower Frame
process_name = 'LowerFrame_1'
event_note='Make a corner junction between aTslot32con and Tslot16 by a M6x25cyl screw.'

action = 'work'
# amount = 0.05
agent = 'one'

cur_pros = process_data[f'{process_name}']

effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.05

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# PROCESS wrap
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# LowerFrame 1
# Output event 1
# Define output event produce for LowerFrame
process_name = 'LowerFrame_1'
event_note='produce LowerFrame'

action = 'produce'
amount = 1
resource = 'lower_frame'
agent = 'two'

cur_pros = process_data[f'{process_name}']

res_data['lower_frame_res'] = {
    "res_ref_id": f'lower_frame-{random.randint(0, 10000)}',
    "name": 'lower_frame',
    "spec_id": res_spec_data['lower_frame']['id']
}
cur_res = res_data[f'{resource}_res']

# for produce events, note new_res=cur_res parameter is required
event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, new_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# LowerFrame 2
# Input event 1
# Define input event accept for LowerFrame 2
process_name = 'LowerFrame_2'
event_note='accept LowerFrame to be modified'

action = 'accept'
amount = 1
resource = 'lower_frame'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']


event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# LowerFrame 2
# Input event 2
# Define input consume 4 x M6nutplates event for LowerFrame Assembly: 
process_name = 'LowerFrame_2'
event_note='Input consume 4 x M6nutplate event for Lower Frame Assembly'

action = 'consume'
amount = 4
resource = 'M6nutplate'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# LowerFrame 2
# Input event 3
# Define input event work of the Lower Frame
process_name = 'LowerFrame_2'
event_note='Insert therefore two M6nutplates into the upper slot and one into the lower slot underneath and one into the outer slot as usual.'

action = 'work'
amount = 0.05
agent = 'two'

cur_pros = process_data[f'{process_name}']

effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = amount

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# PROCESS wrap
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# LowerFrame 2
# Output event 1
# Define output modify Lower Frame
process_name = 'LowerFrame_2'
event_note='modify LowerFrame_2'

action = 'modify'
amount = 1
resource = 'lower_frame'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']


event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# LowerFrame 3
# Input event 1
# Define input event accept for Lower Frame 3
process_name = 'LowerFrame_3'
event_note='accept LowerFrame to be modified'

action = 'accept'
amount = 1
resource = 'lower_frame'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']


event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# LowerFrame 3
# Input event 2
# Define input consume 1 x M6nutplates event for LowerFrame Assembly: 
process_name = 'LowerFrame_3'
event_note='Input consume 1 x M6nutplate event for Lower Frame Assembly'

action = 'consume'
amount = 1
resource = 'M6nutplate'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# EVENT
# LowerFrame 3
# Input event 3
# Define input consume Tslot16 event for LowerFrame Assembly: 
process_name = 'LowerFrame_3'
event_note='Input consume Tslot16 event for Lower Frame Assembly'

action = 'consume'
amount = 1
resource = 'Tslot16'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# LowerFrame 3
# Input event 4
# Define input consume M6x25cyl event for LowerFrame Assembly: 
process_name = 'LowerFrame_3'
event_note='Input consume M6x25cyl event for Lower Frame Assembly'

action = 'consume'
amount = 1
resource = 'M6x25cyl'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# LowerFrame 3
# Input event 5
# Define input use Allen Key for LowerFrame Assembly: 
process_name = 'LowerFrame_3'
event_note='Input use Allen Key event for Lower Frame Assembly'

action = 'use'
amount = 1
resource = 'Allen_Key'
agent = 'one'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

#####
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
# effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.03


event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# LowerFrame 3
# Input event 6
# Define input event work of the Lower Frame
process_name = 'LowerFrame_3'
event_note='Before connecting the second Tslot16 beam you must insert a nutplate into the outer slot of the long Tslot32con, which will be enclosed by the M6x25cyl during the corner connection.'

action = 'work'
amount = 0.05
agent = 'one'

cur_pros = process_data[f'{process_name}']

effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = amount

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# PROCESS wrap
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# LowerFrame 3
# Output event 1
# Define output modify Lower Frame
process_name = 'LowerFrame_3'
event_note='modify LowerFrame_3'

action = 'modify'
amount = 1
resource = 'lower_frame'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']


event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# LowerFrame 4
# Input event 1
# Define input event accept for Lower Frame
process_name = 'LowerFrame_4'
event_note='accept LowerFrame to be modified'

action = 'accept'
amount = 1
resource = 'lower_frame'
agent = 'two'


cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# LowerFrame 4
# Input event 2
# Define input consume 4 x M6nutplates event for LowerFrame Assembly: 
process_name = 'LowerFrame_4'
event_note='Input consume 4 x M6nutplate event for Lower Frame Assembly'

action = 'consume'
amount = 4
resource = 'M6nutplate'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# EVENT
# LowerFrame 4
# Input event 3
# Define input consume Tslot32con event for LowerFrame Assembly: 
process_name = 'LowerFrame_4'
event_note='Input consume Tslot32con event for Lower Frame Assembly'

action = 'consume'
amount = 1
resource = 'Tslot32con'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# LowerFrame 4
# Input event 4
# Define input consume M6x25cyl event for LowerFrame Assembly: 
process_name = 'LowerFrame_4'
event_note='Input consume M6x25cyl event for Lower Frame Assembly'

action = 'consume'
amount = 2
resource = 'M6x25cyl'
agent = 'two'


cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# LowerFrame 4
# Input event 5
# Define input use Allen Key for LowerFrame Assembly: 
process_name = 'LowerFrame_4'
event_note='Input use Allen Key event for Lower Frame Assembly'

action = 'use'
amount = 1
resource = 'Allen_Key'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

#####
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
# effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.03


event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# LowerFrame 4
# Input event 6
# Define input event work of the Lower Frame
process_name = 'LowerFrame_4'
event_note='Before connecting the second Tslot16 beam you must insert 2 nutplates into the upper slot, 1 nutplate into the power slot & 1 nutplate into the outer slot of the long Tslot32com, which will be enclosed by the M6x25cyl during the corner connection.'

action = 'work'
amount = 0.05
agent = 'one'

cur_pros = process_data[f'{process_name}']

effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = amount

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# PROCESS wrap
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# LowerFrame 4
# Output event 1
# Define output modify Lower Frame
process_name = 'LowerFrame_4'
event_note='modify LowerFrame_4'

action = 'modify'
amount = 1
resource = 'lower_frame'
agent = 'two'


cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


## FirstPillar

In [ ]:
# First Pillar
# Pillar_1
# Input event 1
# Define input consume Tslot32 event for Pillar Assembly: 
process_name = 'First_Pillar_1'
event_note='Input consume Tslot32 event for Pillar Assembly'

action = 'consume'
amount = 1
resource = 'Tslot32'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# First Pillar
# Pillar_1
# Input event 2
# Define input consume 1 x M6nutplate event for Pillar Assembly: 
process_name = 'First_Pillar_1'
event_note='Input consume 1 x M6nutplate event for Pillar Assembly'

action = 'consume'
amount = 1
resource = 'M6nutplate'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# First Pillar
# Pillar_1
# Input event 3
# Define input consume RAConnector event for Pillar Assembly: 
process_name = 'First_Pillar_1'
event_note='Input consume 1 x RAConnector event for Pillar Assembly'

action = 'consume'
amount = 1
resource = 'RAConnector'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# First Pillar
# Pillar_1
# Input event 4
# Define input consume 1 x M6x12 event for Pillar Assembly: 
process_name = 'First_Pillar_1'
event_note='Input consume 1 x M6x12 event for Pillar Assembly'

action = 'consume'
amount = 1
resource = 'M6x12'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# First Pillar
# Pillar_1
# Input event 5
# Define input use Allen Key for LowerFrame Assembly: 
process_name = 'First_Pillar_1'
event_note='Input use Allen Key event for Lower Frame Assembly'

action = 'use'
amount = 1
resource = 'Allen_Key'
agent = 'one'

cur_res = res_data[f'{resource}_res']
cur_pros = process_data[f'{process_name}']

#####
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
# effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.03


event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# First Pillar
# Pillar_1
# Input event 6
# Define input event work of the Pillar Pillar_1
process_name = 'First_Pillar_1'
event_note='Insert one M6nutplate at one end of a Tslot32 beam and use it to connect one of the RAconnectors to that beam with a M6x12 screw.'

action = 'work'
amount = 0.05
agent = 'two'

cur_pros = process_data[f'{process_name}']

effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = amount

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# PROCESS wrap
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# First Pillar
# Pillar_1
# Output event 1
# Define output produce event of the Pillar
process_name = 'First_Pillar_1'
event_note='produce Pillar'

action = 'produce'
amount = 1
agent = 'two'

produced_resource = 'first_pillar'
produced_resource_spec = 'pillar'

cur_pros = process_data[f'{process_name}']

res_data[f'{produced_resource}_res'] = {
    "res_ref_id": f'{produced_resource}-{random.randint(0, 10000)}',
    "name": f'{produced_resource}',
    "spec_id": res_spec_data[f'{produced_resource_spec}']['id']
}
cur_res = res_data[f'{produced_resource}_res']

# for produce events, note new_res=cur_res parameter is required
event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, new_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# First Pillar
# Pillar_2
# Input event 1
# Define input event accept of the Pillar
process_name = 'First_Pillar_2'
event_note='accept Pillar to be modified'

action = 'accept'
amount = 1
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{produced_resource}_res']


event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# First Pillar
# Pillar_2
# Input event 2
# Define input consume 3 x M6nutplate event for Pillar Assembly: 
process_name = 'First_Pillar_2'
event_note='Input consume 3 x M6nutplate event for Pillar Assembly'

action = 'consume'
amount = 3
resource = 'M6nutplate'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# First Pillar
# Pillar_2
# Input event 3
# Define input consume RAConnector event for Pillar Assembly: 
process_name = 'First_Pillar_2'
event_note='Input consume 1 x RAConnector event for Pillar Assembly'

action = 'consume'
amount = 1
resource = 'RAConnector'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# First Pillar
# Pillar_2
# Input event 4
# Define input use Allen Key for Pillar Assembly: 
process_name = 'First_Pillar_2'
event_note='Input use Allen Key event for Lower Frame Assembly'

action = 'use'
amount = 1
resource = 'Allen_Key'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

#####
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
# effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.03


event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# First Pillar
# Pillar_2
# Input event 5
# Define input event work of the Pillar Pillar_2
process_name = 'First_Pillar_2'
event_note='Insert three M6nutplates more at the other end of the beam and use one of them to connect another RAconnector there.'

action = 'work'
amount = 0.05
agent = 'one'


cur_pros = process_data[f'{process_name}']

effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = amount

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# PROCESS wrap
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# First Pillar
# Pillar_2
# Output event 1
# Define output event modify Pillar
process_name = 'First_Pillar_2'
event_note='modify Pillar'

action = 'modify'
amount = 1
agent = 'two'


cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{produced_resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Second Pillar
# Pillar_1
# Input event 1
# Define input consume Tslot32 event for Pillar Assembly: 
process_name = 'Second_Pillar_1'
event_note='Input consume Tslot32 event for Pillar Assembly'

action = 'consume'
amount = 1
resource = 'Tslot32'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


## SecondPillar

In [ ]:
# Second Pillar
# Pillar_1
# Input event 2
# Define input consume 1 x M6nutplate event for Pillar Assembly: 
process_name = 'Second_Pillar_1'
event_note='Input consume 1 x M6nutplate event for Pillar Assembly'

action = 'consume'
amount = 1
resource = 'M6nutplate'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Second Pillar
# Pillar_1
# Input event 3
# Define input consume RAConnector event for Pillar Assembly: 
process_name = 'Second_Pillar_1'
event_note='Input consume 1 x RAConnector event for Pillar Assembly'

action = 'consume'
amount = 1
resource = 'RAConnector'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Second Pillar
# Pillar_1
# Input event 4
# Define input consume 1 x M6x12 event for Pillar Assembly: 
process_name = 'Second_Pillar_1'
event_note='Input consume 1 x M6x12 event for Pillar Assembly'

action = 'consume'
amount = 1
resource = 'M6x12'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Second Pillar
# Pillar_1
# Input event 5
# Define input use Allen Key for LowerFrame Assembly: 
process_name = 'Second_Pillar_1'
event_note='Input use Allen Key event for Lower Frame Assembly'

action = 'use'
amount = 1
resource = 'Allen_Key'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

#####
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
# effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.03


event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Second Pillar
# Pillar_1
# Input event 6
# Define input event work of the Pillar Pillar_1
process_name = 'Second_Pillar_1'
event_note='Insert one M6nutplate at one end of a Tslot32 beam and use it to connect one of the RAconnectors to that beam with a M6x12 screw.'

action = 'work'
# amount = 0.05
agent = 'two'

cur_pros = process_data[f'{process_name}']
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.05

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# PROCESS wrap
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# Second Pillar
# Pillar_1
# Output event 1
# Define output produce event of the Pillar
process_name = 'Second_Pillar_1'
event_note='produce Pillar'

action = 'produce'
amount = 1
agent = 'two'

produced_resource ='second_pillar'
produced_resource_spec ='pillar'

cur_pros = process_data[f'{process_name}']

res_data[f'{produced_resource}_res'] = {
    "res_ref_id": f'{produced_resource}-{random.randint(0, 10000)}',
    "name": f'{produced_resource}',
    "spec_id": res_spec_data[f'{produced_resource_spec}']['id']
}
cur_res = res_data[f'{produced_resource}_res']

# for produce events, note new_res=cur_res parameter is required
event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, new_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Second Pillar
# Pillar_2
# Input event 1
# Define input event accept of the Pillar
process_name = 'Second_Pillar_2'
event_note='accept Pillar to be modified'

action = 'accept'
amount = 1
agent = 'two'


cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{produced_resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Second Pillar
# Pillar_2
# Input event 2
# Define input consume 3 x M6nutplate event for Pillar Assembly: 
process_name = 'Second_Pillar_2'
event_note='Input consume 3 x M6nutplate event for Pillar Assembly'

action = 'consume'
amount = 3
resource = 'M6nutplate'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Second Pillar
# Pillar_2
# Input event 3
# Define input consume RAConnector event for Pillar Assembly: 
process_name = 'Second_Pillar_2'
event_note='Input consume 1 x RAConnector event for Pillar Assembly'

action = 'consume'
amount = 1
resource = 'RAConnector'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Second Pillar
# Pillar_2
# Input event 4
# Define input use Allen Key for Pillar Assembly: 
process_name = 'Second_Pillar_2'
event_note='Input use Allen Key event for Lower Frame Assembly'

action = 'use'
amount = 1
resource = 'Allen_Key'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

#####
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
# effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.03

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Second Pillar
# Pillar_2
# Input event 5
# Define input event work of the Pillar Pillar_2
process_name = 'Second_Pillar_2'
event_note='Insert three M6nutplates more at the other end of the beam and use one of them to connect another RAconnector there.'

action = 'work'
# amount = 0.05


cur_pros = process_data[f'{process_name}']
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.05

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# PROCESS wrap
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# Second Pillar
# Pillar_2
# Output event 1
# Define output event modify Pillar
process_name = 'Second_Pillar_2'
event_note='modify Pillar'

action = 'modify'
amount = 1
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{produced_resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


## ThirdPillar

In [ ]:
# Third Pillar
# Pillar_1
# Input event 1
# Define input consume Tslot32 event for Pillar Assembly: 
process_name = 'Third_Pillar_1'
event_note='Input consume Tslot32 event for Pillar Assembly'

action = 'consume'
amount = 1
resource = 'Tslot32'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Third Pillar
# Pillar_1
# Input event 2
# Define input consume 1 x M6nutplate event for Pillar Assembly: 
process_name = 'Third_Pillar_1'
event_note='Input consume 1 x M6nutplate event for Pillar Assembly'

action = 'consume'
amount = 1
resource = 'M6nutplate'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Third Pillar
# Pillar_1
# Input event 3
# Define input consume RAConnector event for Pillar Assembly: 
process_name = 'Third_Pillar_1'
event_note='Input consume 1 x RAConnector event for Pillar Assembly'

action = 'consume'
amount = 1
resource = 'RAConnector'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Third Pillar
# Pillar_1
# Input event 4
# Define input consume 1 x M6x12 event for Pillar Assembly: 
process_name = 'Third_Pillar_1'
event_note='Input consume 1 x M6x12 event for Pillar Assembly'

action = 'consume'
amount = 1
resource = 'M6x12'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Third Pillar
# Pillar_1
# Input event 5
# Define input use Allen Key for LowerFrame Assembly: 
process_name = 'Third_Pillar_1'
event_note='Input use Allen Key event for Lower Frame Assembly'

action = 'use'
amount = 1
resource = 'Allen_Key'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

#####
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
# effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.03


event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Third Pillar
# Pillar_1
# Input event 6
# Define input event work of the Pillar Pillar_1
process_name = 'Third_Pillar_1'
event_note='Insert one M6nutplate at one end of a Tslot32 beam and use it to connect one of the RAconnectors to that beam with a M6x12 screw.'

action = 'work'
# amount = 0.05
agent = 'two'

cur_pros = process_data[f'{process_name}']

effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.05

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# PROCESS wrap
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# Third Pillar
# Pillar_1
# Output event 1
# Define output produce event of the Pillar
process_name = 'Third_Pillar_1'
action = 'produce'
event_note='produce Pillar'
amount = 1

agent = 'two'

produced_resource ='third_pillar'
produced_resource_spec = 'pillar'

cur_pros = process_data[f'{process_name}']

res_data[f'{produced_resource}_res'] = {
    "res_ref_id": f'{produced_resource}-{random.randint(0, 10000)}',
    "name": f'{produced_resource}',
    "spec_id": res_spec_data[f'{produced_resource_spec}']['id']
}
cur_res = res_data[f'{produced_resource}_res']

# for produce events, note new_res=cur_res parameter is required
event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, new_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Third Pillar
# Pillar_2
# Input event 1
# Define input event accept of the Pillar
process_name = 'Third_Pillar_2'
event_note='accept Pillar to be modified'

action = 'accept'
amount = 1
agent = 'two'


cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{produced_resource}_res']


event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Third Pillar
# Pillar_2
# Input event 2
# Define input consume 3 x M6nutplate event for Pillar Assembly: 
process_name = 'Third_Pillar_2'
event_note='Input consume 3 x M6nutplate event for Pillar Assembly'

action = 'consume'
amount = 3
resource = 'M6nutplate'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Third Pillar
# Pillar_2
# Input event 3
# Define input consume RAConnector event for Pillar Assembly: 
process_name = 'Third_Pillar_2'
event_note='Input consume 1 x RAConnector event for Pillar Assembly'

action = 'consume'
amount = 1
resource = 'RAConnector'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Third Pillar
# Pillar_2
# Input event 4
# Define input use Allen Key for Pillar Assembly: 
process_name = 'Third_Pillar_2'
event_note='Input use Allen Key event for Lower Frame Assembly'

action = 'use'
amount = 1
resource = 'Allen_Key'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

#####
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
# effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.03


event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Third Pillar
# Pillar_2
# Input event 5
# Define input event work of the Pillar Pillar_2
process_name = 'Third_Pillar_2'
event_note='Insert three M6nutplates more at the other end of the beam and use one of them to connect another RAconnector there.'

action = 'work'
# amount = 0.05
agent = 'two'

cur_pros = process_data[f'{process_name}']
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.05

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# PROCESS wrap
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# Third Pillar
# Pillar_2
# Output event 1
# Define output event modify Pillar
event_note='modify Pillar'
process_name = 'Third_Pillar_2'

action = 'modify'
amount = 1
agent = 'two'


cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{produced_resource}_res']


event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


## ForthPillar

In [ ]:
# Forth Pillar
# Pillar_1
# Input event 1
# Define input consume Tslot32 event for Pillar Assembly: 
process_name = 'Forth_Pillar_1'
event_note='Input consume Tslot32 event for Pillar Assembly'

action = 'consume'
amount = 1
resource = 'Tslot32'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Forth Pillar
# Pillar_1
# Input event 2
# Define input consume 1 x M6nutplate event for Pillar Assembly: 
process_name = 'Forth_Pillar_1'
event_note='Input consume 1 x M6nutplate event for Pillar Assembly'

action = 'consume'
amount = 1
resource = 'M6nutplate'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Forth Pillar
# Pillar_1
# Input event 3
# Define input consume RAConnector event for Pillar Assembly: 
process_name = 'Forth_Pillar_1'
event_note='Input consume 1 x RAConnector event for Pillar Assembly'

action = 'consume'
amount = 1
resource = 'RAConnector'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Forth Pillar
# Pillar_1
# Input event 4
# Define input consume 1 x M6x12 event for Pillar Assembly: 
process_name = 'Forth_Pillar_1'
event_note='Input consume 1 x M6x12 event for Pillar Assembly'

action = 'consume'
amount = 1
resource = 'M6x12'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Forth Pillar
# Pillar_1
# Input event 5
# Define input use Allen Key for LowerFrame Assembly: 
process_name = 'Forth_Pillar_1'
event_note='Input use Allen Key event for Lower Frame Assembly'

action = 'use'
amount = 1
resource = 'Allen_Key'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

#####
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
# effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.03


event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Forth Pillar
# Pillar_1
# Input event 6
# Define input event work of the Pillar Pillar_1
process_name = 'Forth_Pillar_1'
event_note='Insert one M6nutplate at one end of a Tslot32 beam and use it to connect one of the RAconnectors to that beam with a M6x12 screw.'

action = 'work'
amount = 0.05
agent = 'two'


cur_pros = process_data[f'{process_name}']

effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = amount

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# PROCESS wrap
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# Forth Pillar
# Pillar_1
# Output event 1
# Define output produce event of the Pillar
process_name = 'Forth_Pillar_1'
event_note='produce Pillar'

action = 'produce'
amount = 1

produced_resource ='forth_pillar'
produced_resource_spec ='pillar'

cur_pros = process_data[f'{process_name}']

res_data[f'{produced_resource}_res'] = {
    "res_ref_id": f'{produced_resource}-{random.randint(0, 10000)}',
    "name": f'{produced_resource}',
    "spec_id": res_spec_data[f'{produced_resource_spec}']['id']
}
cur_res = res_data[f'{produced_resource}_res']

# for produce events, note new_res=cur_res parameter is required
event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, new_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Forth Pillar
# Pillar_2
# Input event 1
# Define input event accept of the Pillar
process_name = 'Forth_Pillar_2'
event_note='accept Pillar to be modified'

action = 'accept'
amount = 1
resource = 'forth_pillar'
agent = 'two'


cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Forth Pillar
# Pillar_2
# Input event 2
# Define input consume 3 x M6nutplate event for Pillar Assembly: 
process_name = 'Forth_Pillar_2'
event_note='Input consume 3 x M6nutplate event for Pillar Assembly'

action = 'consume'
amount = 3
resource = 'M6nutplate'
agent = 'two'

cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Forth Pillar
# Pillar_2
# Input event 3
# Define input consume RAConnector event for Pillar Assembly: 
process_name = 'Forth_Pillar_2'
event_note='Input consume 1 x RAConnector event for Pillar Assembly'

action = 'consume'
amount = 1
resource = 'RAConnector'
agent = 'two'


cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Forth Pillar
# Pillar_2
# Input event 4
# Define input use Allen Key for Pillar Assembly: 
process_name = 'Forth_Pillar_2'
event_note='Input use Allen Key event for Lower Frame Assembly'

action = 'use'
amount = 1
resource = 'Allen_Key'
agent = 'two'


cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{resource}_res']

#####
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
# effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.03


event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Forth Pillar
# Pillar_2
# Input event 5
# Define input event work of the Pillar Pillar_2
process_name = 'Forth_Pillar_2'
event_note='Insert three M6nutplates more at the other end of the beam and use one of them to connect another RAconnector there.'

action = 'work'
# amount = 0.05
agent = 'one'


cur_pros = process_data[f'{process_name}']
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.05

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# PROCESS wrap
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# Forth Pillar
# Pillar_2
# Output event 1
# Define output event modify Pillar
process_name = 'Forth_Pillar_2'
event_note='modify Pillar'

action = 'modify'
amount = 1
agent = 'two'


cur_pros = process_data[f'{process_name}']

cur_res = res_data[f'{produced_resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


## BoxFrame

In [ ]:
# First Pillar to Box Frame connection
# First_Pillar_Box_Frame_Connect
# Input event 1
# Define input event accept Lower Frame
process_name = 'First_Pillar_Box_Frame_Connect'
event_note='consume Lower Frame for adding Pillar'

action = 'consume'
amount = 1
resource = 'lower_frame'
agent = 'two'


cur_res = res_data[f'{resource}_res']

cur_pros = process_data[f'{process_name}']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# First Pillar to Box Frame connection
# First_Pillar_Box_Frame_Connect
# Input event 2
# Define input event consume first_pillar
process_name = 'First_Pillar_Box_Frame_Connect'
event_note='consume Pillar into Lower Frame'

action = 'consume'
resource = 'first_pillar'
amount = 1
agent = 'two'


cur_res = res_data[f'{resource}_res']

cur_pros = process_data[f'{process_name}']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# First Pillar to Box Frame connection
# First_Pillar_Box_Frame_Connect
# Input event 3
# Define input event consume M6nutplate
process_name = 'First_Pillar_Box_Frame_Connect'
event_note='consume M6nutplate into Box Frame'

action = 'consume'
resource = 'M6nutplate'
amount = 1
agent = 'two'

cur_res = res_data[f'{resource}_res']

cur_pros = process_data[f'{process_name}']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# First Pillar to Box Frame connection
# First_Pillar_Box_Frame_Connect
# Input event 4
# Define input event consume M6x12 screw
process_name = 'First_Pillar_Box_Frame_Connect'
event_note='consume M6nutplate into Lower Frame'

action = 'consume'
resource = 'M6x12'
amount = 1
agent = 'two'


cur_res = res_data[f'{resource}_res']

cur_pros = process_data[f'{process_name}']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# First Pillar to Box Frame connection
# First_Pillar_Box_Frame_Connect
# Input event 5
# Define input use Allen Key for Pillar to Box Frame connection
process_name = 'First_Pillar_Box_Frame_Connect'
event_note='Input use Allen Key event for Pillar to Lower Frame connection'

action = 'use'
resource = 'Allen_Key'
amount = 1
agent = 'two'


cur_res = res_data[f'{resource}_res']
cur_pros = process_data[f'{process_name}']

#####
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
# effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.03


event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# First Pillar to Box Frame connection
# First_Pillar_Box_Frame_Connect
# Input event 6
# Define input event work for Pillar to Lower Frame connection
process_name = 'First_Pillar_Box_Frame_Connect'
event_note='Insert a M6nutplate into the upper slot of the lower frame subassembly and connect one pillar subassembly to it with a M6x12 screw.'

action = 'work'
amount = 0.05
agent = 'one'


cur_pros = process_data[f'{process_name}']

effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = amount

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# PROCESS wrap
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# First Pillar to Box Frame connection
# First_Pillar_Box_Frame_Connect
# Output event 1
# Define output event produce Box Frame
process_name = 'First_Pillar_Box_Frame_Connect'
event_note='produce Box Frame with First Pillar'

action = 'produce'
# resource = 'box_frame'
amount = 1
agent = 'two'

cur_pros = process_data[f'{process_name}']

produced_resource ='box_frame'
produced_resource_spec ='box_frame'

res_data[f'{produced_resource}_res'] = {
    "res_ref_id": f'{produced_resource}-{random.randint(0, 10000)}',
    "name": f'{produced_resource}',
    "spec_id": res_spec_data[f'{produced_resource_spec}']['id']
}
cur_res = res_data[f'{produced_resource}_res']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, new_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Second Pillar to Box Frame connection
# Second_Pillar_Box_Frame_Connect
# Input event 1
# Define input event accept Box Frame
process_name = 'Second_Pillar_Box_Frame_Connect'
event_note='accept Box Frame for adding Pillar'

action = 'accept'
amount = 1
resource = 'box_frame'
agent = 'two'


cur_res = res_data[f'{resource}_res']

cur_pros = process_data[f'{process_name}']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Second Pillar to Box Frame connection
# Second_Pillar_Box_Frame_Connect
# Input event 2
# Define input event accept Lower Frame
process_name = 'Second_Pillar_Box_Frame_Connect'
event_note='consume Pillar into Box Frame'

action = 'consume'
resource = 'second_pillar'
amount = 1
agent = 'two'


cur_res = res_data[f'{resource}_res']

cur_pros = process_data[f'{process_name}']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Second Pillar to Box Frame connection
# Second_Pillar_Box_Frame_Connect
# Input event 3
# Define input event consume M6nutplate
process_name = 'Second_Pillar_Box_Frame_Connect'
event_note='consume M6nutplate into Lower Frame'

action = 'consume'
resource = 'M6nutplate'
amount = 1
agent = 'two'

cur_res = res_data[f'{resource}_res']

cur_pros = process_data[f'{process_name}']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Second Pillar to Box Frame connection
# Second_Pillar_Box_Frame_Connect
# Input event 4
# Define input event consume M6x12 screw
process_name = 'Second_Pillar_Box_Frame_Connect'
event_note='consume M6nutplate into Lower Frame'

action = 'consume'
resource = 'M6x12'
amount = 1
agent = 'two'


cur_res = res_data[f'{resource}_res']

cur_pros = process_data[f'{process_name}']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Second Pillar to Box Frame connection
# Second_Pillar_Box_Frame_Connect
# Input event 5
# Define input use Allen Key for Pillar to Box Frame connection
process_name = 'Second_Pillar_Box_Frame_Connect'
event_note='Input use Allen Key event for Pillar to Lower Frame connection'

action = 'use'
resource = 'Allen_Key'
amount = 1
agent = 'two'


cur_res = res_data[f'{resource}_res']
cur_pros = process_data[f'{process_name}']

#####
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
# effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.03


event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Second Pillar to Box Frame connection
# Second_Pillar_Box_Frame_Connect
# Input event 6
# Define input event work for Pillar to Box Frame connection
process_name = 'Second_Pillar_Box_Frame_Connect'
event_note='Insert a M6nutplate into the upper slot of the lower frame subassembly and connect one pillar subassembly to it with a M6x12 screw.'

action = 'work'
amount = 0.05
agent = 'one'


cur_pros = process_data[f'{process_name}']

effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = amount

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# PROCESS wrap
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# Second Pillar to Box Frame connection
# Second_Pillar_Box_Frame_Connect
# Output event 1
# Define output event modify Box Frame
process_name = 'Second_Pillar_Box_Frame_Connect'
event_note='modify Lower Frame attaching Pillar'

action = 'modify'
resource = 'box_frame'
amount = 1
agent = 'two'


cur_res = res_data[f'{resource}_res']

cur_pros = process_data[f'{process_name}']


event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Third Pillar to Box Frame connection
# Third_Pillar_Box_Frame_Connect
# Input event 1
# Define input event accept Box Frame
process_name = 'Third_Pillar_Box_Frame_Connect'
event_note='accept Box Frame for adding Pillar'

action = 'accept'
amount = 1
resource = 'box_frame'
agent = 'two'


cur_res = res_data[f'{resource}_res']

cur_pros = process_data[f'{process_name}']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Third Pillar to Box Frame connection
# Third_Pillar_Box_Frame_Connect
# Input event 2
# Define input event consume Pillar for Box Frame
process_name = 'Third_Pillar_Box_Frame_Connect'
event_note='consume Pillar into Box Frame'

action = 'consume'
resource = 'third_pillar'
amount = 1
agent = 'two'


cur_res = res_data[f'{resource}_res']

cur_pros = process_data[f'{process_name}']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Third Pillar to Box Frame connection
# Third_Pillar_Box_Frame_Connect
# Input event 3
# Define input event consume M6nutplate
process_name = 'Third_Pillar_Box_Frame_Connect'
event_note='consume M6nutplate into Lower Frame'

action = 'consume'
resource = 'M6nutplate'
amount = 1
agent = 'two'

cur_res = res_data[f'{resource}_res']

cur_pros = process_data[f'{process_name}']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Third Pillar to Box Frame connection
# Third_Pillar_Box_Frame_Connect
# Input event 4
# Define input event consume M6x12 screw
process_name = 'Third_Pillar_Box_Frame_Connect'
event_note='consume M6nutplate into Box Frame'

action = 'consume'
resource = 'M6x12'
amount = 1
agent = 'two'


cur_res = res_data[f'{resource}_res']

cur_pros = process_data[f'{process_name}']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Third Pillar to Box Frame connection
# Third_Pillar_Box_Frame_Connect
# Input event 5
# Define input use Allen Key for Pillar to Box Frame connection
process_name = 'Third_Pillar_Box_Frame_Connect'
event_note='Input use Allen Key event for Pillar to Lower Frame connection'

action = 'use'
resource = 'Allen_Key'
amount = 1
agent = 'two'


cur_res = res_data[f'{resource}_res']
cur_pros = process_data[f'{process_name}']

#####
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
# effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.03


event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Third Pillar to Box Frame connection
# Third_Pillar_Box_Frame_Connect
# Input event 6
# Define input event work for Pillar to Box Frame connection
process_name = 'Third_Pillar_Box_Frame_Connect'
event_note='Insert a M6nutplate into the upper slot of the lower frame subassembly and connect one pillar subassembly to it with a M6x12 screw.'

action = 'work'
amount = 0.05
agent = 'one'


cur_pros = process_data[f'{process_name}']

effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = amount

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# PROCESS wrap
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# Third Pillar to Box Frame connection
# Third_Pillar_Box_Frame_Connect
# Output event 1
# Define output event modify Box Frame
process_name = 'Third_Pillar_Box_Frame_Connect'
event_note='modify Box Frame attaching Pillar'

action = 'modify'
resource = 'box_frame'
amount = 1
agent = 'two'


cur_res = res_data[f'{resource}_res']

cur_pros = process_data[f'{process_name}']


event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Forth Pillar to Box Frame connection
# Forth_Pillar_Box_Frame_Connect
# Input event 1
# Define input event accept Box Frame
process_name = 'Forth_Pillar_Box_Frame_Connect'
event_note='accept Box Frame for adding Pillar'

action = 'accept'
amount = 1
resource = 'box_frame'
agent = 'two'


cur_res = res_data[f'{resource}_res']

cur_pros = process_data[f'{process_name}']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Forth Pillar to Box Frame connection
# Forth_Pillar_Box_Frame_Connect
## Input event 2
# Define input event accept Box Frame
process_name = 'Forth_Pillar_Box_Frame_Connect'
event_note='consume Pillar into Box Frame'

action = 'consume'
resource = 'forth_pillar'
amount = 1
agent = 'two'


cur_res = res_data[f'{resource}_res']

cur_pros = process_data[f'{process_name}']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Forth Pillar to Box Frame connection
# Forth_Pillar_Box_Frame_Connect
# Input event 3
# Define input event consume M6nutplate
process_name = 'Forth_Pillar_Box_Frame_Connect'
event_note='consume M6nutplate into Lower Frame'

action = 'consume'
resource = 'M6nutplate'
amount = 1
agent = 'two'

cur_res = res_data[f'{resource}_res']

cur_pros = process_data[f'{process_name}']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Forth Pillar to Box Frame connection
# Forth_Pillar_Box_Frame_Connect
# Input event 4
# Define input event consume M6x12 screw
process_name = 'Forth_Pillar_Box_Frame_Connect'
event_note='consume M6x12 screw into Lower Frame'

action = 'consume'
resource = 'M6x12'
amount = 1
agent = 'two'


cur_res = res_data[f'{resource}_res']

cur_pros = process_data[f'{process_name}']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Forth Pillar to Box Frame connection
# Forth_Pillar_Box_Frame_Connect
# Input event 5
# Define input use Allen Key for Pillar to Box Frame connection
process_name = 'Forth_Pillar_Box_Frame_Connect'
event_note='Input use Allen Key event for Pillar to Box Frame connection'

action = 'use'
resource = 'Allen_Key'
amount = 1
agent = 'two'


cur_res = res_data[f'{resource}_res']
cur_pros = process_data[f'{process_name}']

#####
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
# effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.03


event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Forth Pillar to Box Frame connection
# Forth_Pillar_Box_Frame_Connect
# Input event 6
# Define input event work for Pillar to BoxFrame connection
process_name = 'Forth_Pillar_Box_Frame_Connect'
event_note='Insert a M6nutplate into the upper slot of the lower frame subassembly and connect one pillar subassembly to it with a M6x12 screw.'

action = 'work'
amount = 0.05
agent = 'one'


cur_pros = process_data[f'{process_name}']

effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = amount

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# PROCESS wrap
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# Forth Pillar to Box Frame connection
# Forth_Pillar_Box_Frame_Connect
# Output event 1
# Define output event modify Box Frame
process_name = 'Forth_Pillar_Box_Frame_Connect'
event_note='modify Box Frame attaching Pillar'

action = 'modify'
resource = 'box_frame'
amount = 1
agent = 'two'


cur_res = res_data[f'{resource}_res']

cur_pros = process_data[f'{process_name}']


event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Box Frame Side Cover Nutplate Insertion
# Box_Frame_Side_Cover_Nutplates_Insertion
# Input event 1
# Define input event accept Box Frame
process_name = 'Box_Frame_Side_Cover_Nutplates_Insertion'
event_note='accept Lower Frame for adding Side Cover Nutplates'

action = 'accept'
amount = 1
resource = 'box_frame'
agent = 'two'


cur_res = res_data[f'{resource}_res']

cur_pros = process_data[f'{process_name}']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Box Frame Side Cover Nutplate Insertion
# Box_Frame_Side_Cover_Nutplates_Insertion
# Input event 2
# Define input event consume 8 x Nutplates
process_name = 'Box_Frame_Side_Cover_Nutplates_Insertion'
event_note='consume M6nutplates into Lower Frame'

action = 'consume'
resource = 'M6nutplate'
amount = 8
agent = 'two'

cur_res = res_data[f'{resource}_res']

cur_pros = process_data[f'{process_name}']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Box Frame Side Cover Nutplate Insertion
# Box_Frame_Side_Cover_Nutplates_Insertion
# Input event 3
# Define input event work for Side Cover Nutplate Insertion
process_name = 'Box_Frame_Side_Cover_Nutplates_Insertion'
event_note='Insert two M6nutplates in each of the outer slots of the large sides.'

action = 'work'
amount = 0.05
agent = 'one'


cur_pros = process_data[f'{process_name}']

effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = amount

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# PROCESS wrap
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# Box Frame Side Cover Nutplate Insertion
# Box_Frame_Side_Cover_Nutplates_Insertion
# Output event 1
# Define output event modify Box Frame
process_name = 'Box_Frame_Side_Cover_Nutplates_Insertion'
event_note='modify Lower Frame inserting Side Cover Nutplates'

action = 'modify'
resource = 'box_frame'
amount = 1
agent = 'two'


cur_res = res_data[f'{resource}_res']

cur_pros = process_data[f'{process_name}']


event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Box Frame Front Back Cover Nutplate Insertion
# Box_Frame_Front_Back_Cover_Nutplates_Insertion
# Input event 1
# Define input event accept Box Frame
process_name = 'Box_Frame_Front_Back_Cover_Nutplates_Insertion'
event_note='accept Lower Frame for adding Side Cover Nutplates'

action = 'accept'
amount = 1
resource = 'box_frame'
agent = 'two'


cur_res = res_data[f'{resource}_res']

cur_pros = process_data[f'{process_name}']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Box Frame Front Back Cover Nutplate Insertion
# Box_Frame_Front_Back_Cover_Nutplates_Insertion
# Input event 2
# Define input event consume 8 x Nutplates
process_name = 'Box_Frame_Front_Back_Cover_Nutplates_Insertion'
event_note='consume M6nutplates into Lower Frame'

action = 'consume'
resource = 'M6nutplate'
amount = 8
agent = 'two'

cur_res = res_data[f'{resource}_res']

cur_pros = process_data[f'{process_name}']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Box Frame Front Back Cover Nutplate Insertion
# Box_Frame_Front_Back_Cover_Nutplates_Insertion
# Input event 3
# Define input event work for Front/Back Cover Nutplate Insertion
process_name = 'Box_Frame_Front_Back_Cover_Nutplates_Insertion'
event_note='Insert two M6nutplates in each of the outer slots of the front and back sides.'

action = 'work'
amount = 0.05
agent = 'one'


cur_pros = process_data[f'{process_name}']

effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = amount

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# PROCESS wrap
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# Box Frame Front Back Cover Nutplate Insertion
# Box_Frame_Front_Back_Cover_Nutplates_Insertion
# Output event 1
# Define output event modify Lower Frame
process_name = 'Box_Frame_Front_Back_Cover_Nutplates_Insertion'
event_note='modify Box Frame inserting Front/Back Cover Nutplates'

action = 'modify'
resource = 'box_frame'
amount = 1
agent = 'two'


cur_res = res_data[f'{resource}_res']

cur_pros = process_data[f'{process_name}']


event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Box Frame Mountplate Nutplate Insertion
# Box_Frame_Mountplate_Nutplates_Insertion
# Input event 1
# Define input event accept Lower Frame
process_name = 'Box_Frame_Mountplate_Nutplates_Insertion'
event_note='accept Box Frame for adding Side Cover Nutplates'

action = 'accept'
amount = 1
resource = 'box_frame'
agent = 'two'


cur_res = res_data[f'{resource}_res']

cur_pros = process_data[f'{process_name}']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Box Frame Mountplate Nutplate Insertion
# Box_Frame_Mountplate_Nutplates_Insertion
# Input event 2
# Define input event consume 4 x Nutplates
process_name = 'Box_Frame_Mountplate_Nutplates_Insertion'
event_note='consume M6nutplates into Lower Frame'

action = 'consume'
resource = 'M6nutplate'
amount = 4
agent = 'two'

cur_res = res_data[f'{resource}_res']

cur_pros = process_data[f'{process_name}']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Box Frame Mountplate Nutplate Insertion
# Box_Frame_Mountplate_Nutplates_Insertion
## Input event 3
# Define input event work for Mountplate Nutplate Insertion
process_name = 'Box_Frame_Mountplate_Nutplates_Insertion'
event_note='Insert two M6nutplates in each of the inner slots of one small side. This defines, which will later become the front side.'

action = 'work'
amount = 0.05
agent = 'one'


cur_pros = process_data[f'{process_name}']

effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = amount

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# PROCESS wrap
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# Box Frame Mountplate Nutplate Insertion
# Box_Frame_Mountplate_Nutplates_Insertion
# Output event 1
# Define output event modify Lower Frame
process_name = 'Box_Frame_Mountplate_Nutplates_Insertion'
event_note='modify Lower Frame inserting Mountplates Nutplates'

action = 'modify'
resource = 'box_frame'
amount = 1
agent = 'two'


cur_res = res_data[f'{resource}_res']

cur_pros = process_data[f'{process_name}']


event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Box Frame Upper Frame Connection
# Box_Upper_Frame_Connection
# Input event 1
# Define input event accept Box Frame
process_name = 'Box_Upper_Frame_Connection'
event_note='accept Box Frame'

action = 'accept'
resource = 'box_frame'
amount = 1
agent = 'two'

cur_res = res_data[f'{resource}_res']

cur_pros = process_data[f'{process_name}']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Box Frame Upper Frame Connection
# Box_Upper_Frame_Connection
# Input event 2
# Define input event consume Upper Frame
process_name = 'Box_Upper_Frame_Connection'
event_note='consume Upper Frame into Box Frame'

action = 'consume'
resource = 'upper_frame'
amount = 1
agent = 'two'

cur_res = res_data[f'{resource}_res']

cur_pros = process_data[f'{process_name}']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Box Frame Upper Frame Connection
# Box_Upper_Frame_Connection
# Input event 3
# Define input event consume 4 x Nutplates
process_name = 'Box_Upper_Frame_Connection'
event_note='consume 4 x M6nutplates into Box Frame'

action = 'consume'
resource = 'M6nutplate'
amount = 4
agent = 'two'

cur_res = res_data[f'{resource}_res']

cur_pros = process_data[f'{process_name}']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Box Frame Upper Frame Connection
# Box_Upper_Frame_Connection
# Input event 4
# Define input event consume 4 x M6x12
process_name = 'Box_Upper_Frame_Connection'
event_note='consume 4 x M6x12 into Box Frame'

action = 'consume'
resource = 'M6x12'
amount = 4
agent = 'two'

cur_res = res_data[f'{resource}_res']

cur_pros = process_data[f'{process_name}']

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Box Frame Upper Frame Connection
# Box_Upper_Frame_Connection
# Input event 5
# Define input use Allen Key for Pillar to Lower Frame connection
process_name = 'Box_Upper_Frame_Connection'
event_note='Input use Allen Key event for Lower Frame Upper Frame Connection'

action = 'use'
resource = 'Allen_Key'
amount = 1
agent = 'two'


cur_res = res_data[f'{resource}_res']
cur_pros = process_data[f'{process_name}']

#####
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
# effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.03


event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Box Frame Upper Frame Connection
# Box_Upper_Frame_Connection
# Input event 6
# Define input event work for Mountplate Nutplate Insertion
process_name = 'Box_Upper_Frame_Connection'
event_note='Place the upper frame on top and insert the four M6nutplates into the lower slot of each corner. Then make the connection to the Raconnectors of the pillars with the M6x12 screws.'

action = 'work'
amount = 0.05
agent = 'one'


cur_pros = process_data[f'{process_name}']

effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = amount

event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# PROCESS wrap
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# Box Frame Upper Frame Connection
# Box_Upper_Frame_Connection
# Output event 1
# Define output modify event of the BoxFrame
process_name = 'Box_Upper_Frame_Connection'
event_note='modify Box Frame'

action = 'modify'
amount = 1
resource = 'box_frame'
agent = 'two'


cur_res = res_data[f'{resource}_res']

cur_pros = process_data[f'{process_name}']


event_id, ts = create_event(users_data[f'{agent}'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


## Groups processes

In [ ]:
# Create the box_frame process group to hold subcomponent process groups
name = note = processgrp = process = processgrp_id = None
name = "assemble_box_frame"
note = "Groups box frame subcomponent groups: upper frame, lower frame & four pillars"
agent = 'one'


get_processgrp(name, users_data[f'{agent}'], note, processgrp_data, processgrp_id=processgrp_id, endpoint=ENDPOINT)

In [ ]:
# UPPER_FRAME
# Create the process group to hold the processes
group_name = note = processgrp = process = processgrp_id = agent = None

group_name = "assemble_upper_frame"
note = "Groups assembly events of the upper frame"
parent_group = "assemble_box_frame"
agent = 'one'

processgrp_id = processgrp_data[f'{parent_group}']['id']

get_processgrp(group_name, users_data[f'{agent}'], note, processgrp_data, processgrp_id=processgrp_id, endpoint=ENDPOINT)


In [ ]:
# UPPER FRAME
# Insert the processes in the group
group_name = note = processgrp = process = processgrp_id = None

agent = 'one'
group_name = "assemble_upper_frame"
process_list = ['UpperFrame_1', 'UpperFrame_2', 'UpperFrame_3', 'UpperFrame_4']


processgrp = processgrp_data[f'{group_name}']

for process in process_list:
    insert_procingrp(users_data[f'{agent}'], processgrp, process_data[f'{process}'], endpoint=ENDPOINT)


In [ ]:
# LOWER FRAME
# Create the process group to hold the processes
group_name = note = processgrp = process = processgrp_id = agent = None

group_name = "assemble_lower_frame"
note = "Groups assembly events of the lower frame"
parent_group = "assemble_box_frame"
agent = 'one'

processgrp_id = processgrp_data[f'{parent_group}']['id']

get_processgrp(group_name, users_data[f'{agent}'], note, processgrp_data, processgrp_id=processgrp_id, endpoint=ENDPOINT)


In [ ]:
# LOWER FRAME
# Insert the processes in the group
group_name = note = processgrp = process = processgrp_id = None

agent = 'one'
group_name = "assemble_lower_frame"
process_list = ['LowerFrame_1', 'LowerFrame_2', 'LowerFrame_3', 'LowerFrame_4']


processgrp = processgrp_data[f'{group_name}']

for process in process_list:
    insert_procingrp(users_data[f'{agent}'], processgrp, process_data[f'{process}'], endpoint=ENDPOINT)


In [ ]:
# FIRST PILLAR
# Create the process group to hold the processes
group_name = note = processgrp = process = processgrp_id = agent = None

group_name = "assemble_first_pillar"
note = "Groups assembly events of the first pillar"
parent_group = "assemble_box_frame"
agent = 'one'

processgrp_id = processgrp_data[f'{parent_group}']['id']

get_processgrp(group_name, users_data[f'{agent}'], note, processgrp_data, processgrp_id=processgrp_id, endpoint=ENDPOINT)


In [ ]:
# FIRST PILLAR
# Insert the processes in the group
group_name = note = processgrp = process = processgrp_id = None

agent = 'one'
group_name = "assemble_first_pillar"
process_list = ['First_Pillar_1', 'First_Pillar_2']


processgrp = processgrp_data[f'{group_name}']

for process in process_list:
    insert_procingrp(users_data[f'{agent}'], processgrp, process_data[f'{process}'], endpoint=ENDPOINT)


In [ ]:
# SECOND PILLAR
# Create the process group to hold the processes
group_name = note = processgrp = process = processgrp_id = agent = None

group_name = "assemble_second_pillar"
note = "Groups assembly events of the second pillar"
parent_group = "assemble_box_frame"
agent = 'one'

processgrp_id = processgrp_data[f'{parent_group}']['id']

get_processgrp(group_name, users_data[f'{agent}'], note, processgrp_data, processgrp_id=processgrp_id, endpoint=ENDPOINT)


In [ ]:
# SECOND PILLAR
# Insert the processes in the group
group_name = note = processgrp = process = processgrp_id = None

agent = 'one'
group_name = "assemble_second_pillar"
process_list = ['Second_Pillar_1', 'Second_Pillar_2']


processgrp = processgrp_data[f'{group_name}']

for process in process_list:
    insert_procingrp(users_data[f'{agent}'], processgrp, process_data[f'{process}'], endpoint=ENDPOINT)


In [ ]:
# THIRD PILLAR
# Create the process group to hold the processes
group_name = note = processgrp = process = processgrp_id = agent = None

group_name = "assemble_third_pillar"
note = "Groups assembly events of the third pillar"
parent_group = "assemble_box_frame"
agent = 'one'

processgrp_id = processgrp_data[f'{parent_group}']['id']

get_processgrp(group_name, users_data[f'{agent}'], note, processgrp_data, processgrp_id=processgrp_id, endpoint=ENDPOINT)


In [ ]:
# THIRD PILLAR
# Insert the processes in the group
group_name = note = processgrp = process = processgrp_id = None

agent = 'one'
group_name = "assemble_third_pillar"
process_list = ['Third_Pillar_1', 'Third_Pillar_2']


processgrp = processgrp_data[f'{group_name}']

for process in process_list:
    insert_procingrp(users_data[f'{agent}'], processgrp, process_data[f'{process}'], endpoint=ENDPOINT)


In [ ]:
# FORTH PILLAR
# Create the process group to hold the processes
group_name = note = processgrp = process = processgrp_id = agent = None

group_name = "assemble_forth_pillar"
note = "Groups assembly events of the forth pillar"
parent_group = "assemble_box_frame"
agent = 'one'

processgrp_id = processgrp_data[f'{parent_group}']['id']

get_processgrp(group_name, users_data[f'{agent}'], note, processgrp_data, processgrp_id=processgrp_id, endpoint=ENDPOINT)


In [ ]:
# FORTH PILLAR
# Insert the processes in the group
group_name = note = processgrp = process = processgrp_id = None

agent = 'one'
group_name = "assemble_forth_pillar"
process_list = ['Forth_Pillar_1', 'Forth_Pillar_2']


processgrp = processgrp_data[f'{group_name}']

for process in process_list:
    insert_procingrp(users_data[f'{agent}'], processgrp, process_data[f'{process}'], endpoint=ENDPOINT)


In [ ]:
# BOX FRAME PILLAR ATTACH
# Create the process group to hold the processes
group_name = note = processgrp = process = processgrp_id = agent = None

group_name = "attach_pillars_to_box_frame"
note = "Groups assembly attaching 4 pillars to box_frame"
parent_group = "assemble_box_frame"
agent = 'one'

processgrp_id = processgrp_data[f'{parent_group}']['id']

get_processgrp(group_name, users_data[f'{agent}'], note, processgrp_data, processgrp_id=processgrp_id, endpoint=ENDPOINT)


In [ ]:
# BOX FRAME PILLAR ATTACH
# Insert the processes in the group
group_name = note = processgrp = process = processgrp_id = None

agent = 'one'
group_name = "attach_pillars_to_box_frame"
process_list = ['First_Pillar_Box_Frame_Connect','Second_Pillar_Box_Frame_Connect','Third_Pillar_Box_Frame_Connect','Forth_Pillar_Box_Frame_Connect']


processgrp = processgrp_data[f'{group_name}']

for process in process_list:
    insert_procingrp(users_data[f'{agent}'], processgrp, process_data[f'{process}'], endpoint=ENDPOINT)


In [ ]:
# BOX FRAME NUTPLATE INSERTION
# Create the process group to hold the processes
group_name = note = processgrp = process = processgrp_id = agent = None

group_name = "insert_nutplates_in_box_frame"
note = "Groups assembly inserting nutplates into box_frame"
parent_group = "assemble_box_frame"
agent = 'one'

processgrp_id = processgrp_data[f'{parent_group}']['id']

get_processgrp(group_name, users_data[f'{agent}'], note, processgrp_data, processgrp_id=processgrp_id, endpoint=ENDPOINT)


In [ ]:
# BOX FRAME NUTPLATE INSERTION
# Insert the processes in the group
group_name = note = processgrp = process = processgrp_id = None

agent = 'one'
group_name = "insert_nutplates_in_box_frame"
process_list = ['Box_Frame_Side_Cover_Nutplates_Insertion','Box_Frame_Front_Back_Cover_Nutplates_Insertion','Box_Frame_Mountplate_Nutplates_Insertion']


processgrp = processgrp_data[f'{group_name}']

for process in process_list:
    insert_procingrp(users_data[f'{agent}'], processgrp, process_data[f'{process}'], endpoint=ENDPOINT)


In [ ]:
# BOX FRAME UPPER FRAME ATTACH
# Create the process group to hold the processes
# when these are uncommentde, visualisation seems to break?
group_name = note = processgrp = process = processgrp_id = agent = None

group_name = "attach_upper_frame_to_box_frame"
note = "Groups assembly attaching upper frame to box frame"
parent_group = "assemble_box_frame"
agent = 'one'

processgrp_id = processgrp_data[f'{parent_group}']['id']

get_processgrp(group_name, users_data[f'{agent}'], note, processgrp_data, processgrp_id=processgrp_id, endpoint=ENDPOINT)


In [ ]:
# BOX FRAME UPPER FRAME ATTACH
# Insert the processes in the group
group_name = note = processgrp = process = processgrp_id = agent = None

agent = 'one'
group_name = "attach_upper_frame_to_box_frame"
process_list = ['Box_Upper_Frame_Connection']
process = 'Box_Upper_Frame_Connection'


processgrp = processgrp_data[f'{group_name}']

for process in process_list:
    insert_procingrp(users_data[f'{agent}'], processgrp, process_data[f'{process}'], endpoint=ENDPOINT)


## Transfer BoxFrame

In [ ]:
# XXXX Mock Transfer from user two to user one
note='Mock Transfer UpperFrame from two to one'
action = 'transfer'
amount = 1
resource = 'box_frame'

cur_res = res_data[f'{resource}_res']

event_id, ts = make_transfer(users_data['two'], action, note, users_data['one'], amount, cur_res, locs_data, res_spec_data, endpoint=ENDPOINT)
event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
show_data(users_data, locs_data, res_data, units_data, res_spec_data, process_data, event_seq, \
          proposal_data, intent_data, prop_int_data, satisfaction_data, processgrp_data)

In [ ]:
# example of using show functions
# show_resource(users_data['one'], '061Z9FX8M4BJ3M6JMB7MW7VSER', endpoint=ENDPOINT)
# show_proposal(users_data['one'], '061Z9FX9H8PT0JF16KP25VS8R0', endpoint=ENDPOINT)


## Traces

In [ ]:
trace_object = 'upper_frame'

trace_resource = f'{trace_object}_res'
trace_me = res_data[f'{trace_resource}']['id']

print(f"Resource to be traced: {trace_me}")
tot_dpp = []
visited = set()
er_before(trace_me, users_data['two'], dpp_children=tot_dpp, depth=0, visited=visited, endpoint=ENDPOINT)

# Serializing json
json_object = json.dumps(tot_dpp, indent=2)

# print(json_object)
# print(visited)

In [ ]:
be_dpp = get_dpp(trace_me, user_data, endpoint=ENDPOINT)
# print(json.dumps(be_dpp, indent=2))

In [ ]:
trace = trace_query(trace_me, users_data['two'], endpoint=ENDPOINT)
# check consistency between the registered events, the back-end trace and the generated dpp
# XXX DEBUG comment check_traces for now
# check_traces(trace, event_seq, tot_dpp, be_dpp)

In [ ]:
save_traces(f'{USE_CASE}-{trace_object}', tot_dpp, trace, be_dpp, event_seq, processgrp_data)

In [ ]:
labels = []
sources = []
targets = []
values = []
color_nodes = []
color_links = []
assigned = {}
vis_dpp(tot_dpp[0], count=0, assigned=assigned, labels=labels, targets=targets, sources=sources, values=values, color_nodes=color_nodes, color_links=color_links)
sources, targets = consol_trace(assigned, sources, targets)
make_sankey(sources, targets, labels, values, color_nodes, color_links)

In [ ]:
trace_object = 'lower_frame'

trace_resource = f'{trace_object}_res'
trace_me = res_data[f'{trace_resource}']['id']

print(f"Resource to be traced: {trace_me}")
tot_dpp = []
visited = set()
er_before(trace_me, users_data['two'], dpp_children=tot_dpp, depth=0, visited=visited, endpoint=ENDPOINT)

# Serializing json
json_object = json.dumps(tot_dpp, indent=2)

# print(json_object)
# print(visited)

In [ ]:
be_dpp = get_dpp(trace_me, user_data, endpoint=ENDPOINT)
# print(json.dumps(be_dpp, indent=2))

In [ ]:
trace = trace_query(trace_me, users_data['two'], endpoint=ENDPOINT)
# check consistency between the registered events, the back-end trace and the generated dpp
# XXX DEBUG comment check_traces for now
# check_traces(trace, event_seq, tot_dpp, be_dpp)

In [ ]:
save_traces(f'{USE_CASE}-{trace_object}', tot_dpp, trace, be_dpp, event_seq, processgrp_data)

In [ ]:
labels = []
sources = []
targets = []
values = []
color_nodes = []
color_links = []
assigned = {}
vis_dpp(tot_dpp[0], count=0, assigned=assigned, labels=labels, targets=targets, sources=sources, values=values, color_nodes=color_nodes, color_links=color_links)
sources, targets = consol_trace(assigned, sources, targets)
make_sankey(sources, targets, labels, values, color_nodes, color_links)

In [ ]:
trace_object = 'first_pillar'

trace_resource = f'{trace_object}_res'
trace_me = res_data[f'{trace_resource}']['id']

print(f"Resource to be traced: {trace_me}")
tot_dpp = []
visited = set()
er_before(trace_me, users_data['two'], dpp_children=tot_dpp, depth=0, visited=visited, endpoint=ENDPOINT)

# Serializing json
json_object = json.dumps(tot_dpp, indent=2)

# print(json_object)
# print(visited)

In [ ]:
be_dpp = get_dpp(trace_me, user_data, endpoint=ENDPOINT)
# print(json.dumps(be_dpp, indent=2))

In [ ]:
trace = trace_query(trace_me, users_data['two'], endpoint=ENDPOINT)
# check consistency between the registered events, the back-end trace and the generated dpp
# XXX DEBUG comment check_traces for now
# check_traces(trace, event_seq, tot_dpp, be_dpp)

In [ ]:
save_traces(f'{USE_CASE}-{trace_object}', tot_dpp, trace, be_dpp, event_seq, processgrp_data)

In [ ]:
labels = []
sources = []
targets = []
values = []
color_nodes = []
color_links = []
assigned = {}
vis_dpp(tot_dpp[0], count=0, assigned=assigned, labels=labels, targets=targets, sources=sources, values=values, color_nodes=color_nodes, color_links=color_links)
sources, targets = consol_trace(assigned, sources, targets)
make_sankey(sources, targets, labels, values, color_nodes, color_links)

In [ ]:
trace_object = 'second_pillar'

trace_resource = f'{trace_object}_res'
trace_me = res_data[f'{trace_resource}']['id']

print(f"Resource to be traced: {trace_me}")
tot_dpp = []
visited = set()
er_before(trace_me, users_data['two'], dpp_children=tot_dpp, depth=0, visited=visited, endpoint=ENDPOINT)

# Serializing json
json_object = json.dumps(tot_dpp, indent=2)

# print(json_object)
# print(visited)

In [ ]:
be_dpp = get_dpp(trace_me, user_data, endpoint=ENDPOINT)
# print(json.dumps(be_dpp, indent=2))

In [ ]:
trace = trace_query(trace_me, users_data['two'], endpoint=ENDPOINT)
# check consistency between the registered events, the back-end trace and the generated dpp
# XXX DEBUG comment check_traces for now
# check_traces(trace, event_seq, tot_dpp, be_dpp)

In [ ]:
save_traces(f'{USE_CASE}-{trace_object}', tot_dpp, trace, be_dpp, event_seq, processgrp_data)

In [ ]:
labels = []
sources = []
targets = []
values = []
color_nodes = []
color_links = []
assigned = {}
vis_dpp(tot_dpp[0], count=0, assigned=assigned, labels=labels, targets=targets, sources=sources, values=values, color_nodes=color_nodes, color_links=color_links)
sources, targets = consol_trace(assigned, sources, targets)
make_sankey(sources, targets, labels, values, color_nodes, color_links)

In [ ]:
trace_object = 'third_pillar'

trace_resource = f'{trace_object}_res'
trace_me = res_data[f'{trace_resource}']['id']

print(f"Resource to be traced: {trace_me}")
tot_dpp = []
visited = set()
er_before(trace_me, users_data['two'], dpp_children=tot_dpp, depth=0, visited=visited, endpoint=ENDPOINT)

# Serializing json
json_object = json.dumps(tot_dpp, indent=2)

# print(json_object)
# print(visited)

In [ ]:
be_dpp = get_dpp(trace_me, user_data, endpoint=ENDPOINT)
# print(json.dumps(be_dpp, indent=2))

In [ ]:
trace = trace_query(trace_me, users_data['two'], endpoint=ENDPOINT)
# check consistency between the registered events, the back-end trace and the generated dpp
# XXX DEBUG comment check_traces for now
# check_traces(trace, event_seq, tot_dpp, be_dpp)

In [ ]:
save_traces(f'{USE_CASE}-{trace_object}', tot_dpp, trace, be_dpp, event_seq, processgrp_data)

In [ ]:
labels = []
sources = []
targets = []
values = []
color_nodes = []
color_links = []
assigned = {}
vis_dpp(tot_dpp[0], count=0, assigned=assigned, labels=labels, targets=targets, sources=sources, values=values, color_nodes=color_nodes, color_links=color_links)
sources, targets = consol_trace(assigned, sources, targets)
make_sankey(sources, targets, labels, values, color_nodes, color_links)

In [ ]:
trace_object = 'forth_pillar'

trace_resource = f'{trace_object}_res'
trace_me = res_data[f'{trace_resource}']['id']

print(f"Resource to be traced: {trace_me}")
tot_dpp = []
visited = set()
er_before(trace_me, users_data['two'], dpp_children=tot_dpp, depth=0, visited=visited, endpoint=ENDPOINT)

# Serializing json
json_object = json.dumps(tot_dpp, indent=2)

# print(json_object)
# print(visited)

In [ ]:
be_dpp = get_dpp(trace_me, user_data, endpoint=ENDPOINT)
# print(json.dumps(be_dpp, indent=2))

In [ ]:
trace = trace_query(trace_me, users_data['two'], endpoint=ENDPOINT)
# check consistency between the registered events, the back-end trace and the generated dpp
# XXX DEBUG comment check_traces for now
# check_traces(trace, event_seq, tot_dpp, be_dpp)

In [ ]:
save_traces(f'{USE_CASE}-{trace_object}', tot_dpp, trace, be_dpp, event_seq, processgrp_data)

In [ ]:
labels = []
sources = []
targets = []
values = []
color_nodes = []
color_links = []
assigned = {}
vis_dpp(tot_dpp[0], count=0, assigned=assigned, labels=labels, targets=targets, sources=sources, values=values, color_nodes=color_nodes, color_links=color_links)
sources, targets = consol_trace(assigned, sources, targets)
make_sankey(sources, targets, labels, values, color_nodes, color_links)

In [ ]:
# XXXX DEBUG testing viz with upper frame

# trace_me = res_data['first_pillar_res']['id']
# trace_me = res_data['box_frame_res']['id']
# trace_object = 'upper_frame'
# trace_object = 'first_pillar'
trace_object = 'box_frame'

trace_resource = f'{trace_object}_res'
trace_me = res_data[f'{trace_resource}']['id']

print(f"Resource to be traced: {trace_me}")
tot_dpp = []
visited = set()
er_before(trace_me, users_data['two'], dpp_children=tot_dpp, depth=0, visited=visited, endpoint=ENDPOINT)

# Serializing json
json_object = json.dumps(tot_dpp, indent=2)

# print(json_object)
# print(visited)

In [ ]:
be_dpp = get_dpp(trace_me, user_data, endpoint=ENDPOINT)
# print(json.dumps(be_dpp, indent=2))

In [ ]:
trace = trace_query(trace_me, users_data['two'], endpoint=ENDPOINT)
# XXX DEBUG comment check_traces for now
# check_traces(trace, event_seq, tot_dpp, be_dpp)

In [ ]:
save_traces(USE_CASE, tot_dpp, trace, be_dpp, event_seq, processgrp_data)
save_traces(f'{USE_CASE}-{trace_object}', tot_dpp, trace, be_dpp, event_seq, processgrp_data)

In [ ]:
labels = []
sources = []
targets = []
values = []
color_nodes = []
color_links = []
assigned = {}
vis_dpp(tot_dpp[0], count=0, assigned=assigned, labels=labels, targets=targets, sources=sources, values=values, color_nodes=color_nodes, color_links=color_links)
sources, targets = consol_trace(assigned, sources, targets)
make_sankey(sources, targets, labels, values, color_nodes, color_links)